In [28]:
import re

import pandas as pd
import mlflow
from sklearn.metrics import mean_absolute_error

from machine_learning.ml_data import MLData
from machine_learning.settings import TRAIN_YEARS, VALIDATION_YEARS, TEST_YEARS
from machine_learning.data_import import betting_data
from machine_learning.ml_estimators import StackingEstimator

In [2]:
data = MLData(train_years=TRAIN_YEARS, test_years=VALIDATION_YEARS)
data.data

2019-12-11 02:28:23,903 - kedro.io.data_catalog - INFO - Loading data from `model_data` (JSONLocalDataSet)...


/usr/local/lib/python3.6/site-packages/kedro/context/context.py:327: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/app/conf/base', '/app/conf/development'] matching the glob pattern(s): ['credentials*', 'credentials*/**']
  str(exc)


team   oppo_team round_type  \
Adelaide         1991 1           Adelaide    Hawthorn    Regular   
                      2           Adelaide     Carlton    Regular   
                      3           Adelaide      Sydney    Regular   
                      4           Adelaide    Essendon    Regular   
                      5           Adelaide  West Coast    Regular   
...                                    ...         ...        ...   
Western Bulldogs 2019 20  Western Bulldogs    Brisbane    Regular   
                      21  Western Bulldogs    Essendon    Regular   
                      22  Western Bulldogs         GWS    Regular   
                      23  Western Bulldogs    Adelaide    Regular   
                      24  Western Bulldogs         GWS     Finals   

                                      venue prev_match_oppo_team match_id  \
Adelaide         1991 1       Football Park                    0    10128   
                      2       Football Park             Hawthorn    10135   
                      3              S.C.G.              Carlton    10143   
                      4          Windy Hill               Sydney    10147   
                      5             Subiaco             Essendon    10159   
...                                     ...                  ...      ...   
Western Bulldogs 2019 20              Gabba            Fremantle    15578   
                      21          Docklands             Brisbane    15583   
                      22  Sydney Showground             Essendon    15595   
                      23     Eureka Stadium                  GWS    15603   
                      24  Sydney Showground             Adelaide    15608   

                                              date  team_goals  team_behinds  \
Adelaide         1991 1  1991-03-22 03:56:00+00:00          24            11   
                      2  1991-03-31 03:56:00+00:00          12             9   
                      3  1991-04-07 03:05:00+00:00          19            18   
                      4  1991-04-13 03:30:00+00:00           6            11   
                      5  1991-04-21 05:27:00+00:00           9            11   
...                                            ...         ...           ...   
Western Bulldogs 2019 20 2019-08-04 02:58:00+00:00          11            14   
                      21 2019-08-10 03:30:00+00:00          21            11   
                      22 2019-08-18 03:05:00+00:00          19            12   
                      23 2019-08-25 03:30:00+00:00          18            13   
                      24 2019-09-07 03:05:00+00:00           8             7   

                          score  ...  \
Adelaide         1991 1     155  ...   
                      2      81  ...   
                      3     132  ...   
                      4      47  ...   
                      5      65  ...   
...                         ...  ...   
Western Bulldogs 2019 20     80  ...   
                      21    137  ...   
                      22    126  ...   
                      23    121  ...   
                      24     55  ...   

                          oppo_rolling_prev_match_goals_divided_by_rolling_prev_match_goals_plus_rolling_prev_match_behinds  \
Adelaide         1991 1                                                 0.0                                                   
                      2                                                 0.0                                                   
                      3                                                 0.0                                                   
                      4                                                 0.0                                                   
                      5                                                 0.0                                                   
...                                                                     ...     

In [3]:
leg_data = MLData(
    train_years=TRAIN_YEARS,
    test_years=VALIDATION_YEARS,
    data_set='legacy_model_data',
    pipeline='legacy'
)
leg_data.data

2019-12-11 02:29:30,603 - kedro.io.data_catalog - INFO - Loading data from `legacy_model_data` (JSONLocalDataSet)...


team   oppo_team round_type  \
Adelaide         1991 1           Adelaide    Hawthorn    Regular   
                      2           Adelaide     Carlton    Regular   
                      3           Adelaide      Sydney    Regular   
                      4           Adelaide    Essendon    Regular   
                      5           Adelaide  West Coast    Regular   
...                                    ...         ...        ...   
Western Bulldogs 2019 20  Western Bulldogs    Brisbane    Regular   
                      21  Western Bulldogs    Essendon    Regular   
                      22  Western Bulldogs         GWS    Regular   
                      23  Western Bulldogs    Adelaide    Regular   
                      24  Western Bulldogs         GWS     Finals   

                                      venue                      date  \
Adelaide         1991 1       Football Park 1991-03-22 03:56:00+00:00   
                      2       Football Park 1991-03-31 03:56:00+00:00   
                      3              S.C.G. 1991-04-07 03:05:00+00:00   
                      4          Windy Hill 1991-04-13 03:30:00+00:00   
                      5             Subiaco 1991-04-21 05:27:00+00:00   
...                                     ...                       ...   
Western Bulldogs 2019 20              Gabba 2019-08-04 02:58:00+00:00   
                      21          Docklands 2019-08-10 03:30:00+00:00   
                      22  Sydney Showground 2019-08-18 03:05:00+00:00   
                      23     Eureka Stadium 2019-08-25 03:30:00+00:00   
                      24  Sydney Showground 2019-09-07 03:05:00+00:00   

                          team_goals  team_behinds  score  oppo_team_goals  \
Adelaide         1991 1           24            11    155                9   
                      2           12             9     81               15   
                      3           19            18    132               15   
                      4            6            11     47               12   
                      5            9            11     65               19   
...                              ...           ...    ...              ...   
Western Bulldogs 2019 20          11            14     80               14   
                      21          21            11    137                4   
                      22          19            12    126                9   
                      23          18            13    121               13   
                      24           8             7     55               16   

                          oppo_team_behinds  ...  win_odds  oppo_win_odds  \
Adelaide         1991 1                  15  ...      0.00           0.00   
                      2                  14  ...      0.00           0.00   
                      3                  18  ...      0.00           0.00   
                      4                  20  ...      0.00           0.00   
                      5                  16  ...      0.00           0.00   
...                                     ...  ...       ...            ...   
Western Bulldogs 2019 20                 14  ...      3.15           1.36   
                      21                  9  ...      1.68           2.20   
                      22                 11  ...      2.15           1.71   
                      23                  9  ...      1.33           3.35   
                      24                 17  ...      2.00           1.82   

                          line_odds  oppo_line_odds  betting_pred_win  \
Adelaide         1991 1         0.0             0.0               0.0   
                      2         0.0             0.0               0.0   
                      3         0.0             0.0               0.0   
                      4         0.0             0.0               0.0   
                      5         0.0             0.0               0.0   
...                             

In [4]:
# mlflow.create_experiment('initial_stacking_model')
mlflow.set_experiment('initial_stacking_model')

In [5]:
se = StackingEstimator()
se.fit(*data.train_data())

2019-12-11 02:30:43,760 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


/usr/local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


StackingEstimator(name='stacking_estimator',
                  pipeline=Model(name='Model_0', function='predict', n_outputs=1, trainable=True))

In [15]:
def _is_relevant_param(key, value):
    STATIC_TRANSFORMERS = ["onehotencoder", "columnselector", "columndropper", "teammatchtomatchconverter"]
    STATIC_COL_PARAMS = ["cols_to_keep", "match_cols"]
    STATIC_PARAMS = ["verbosity"]
    IRRELEVANT_PARAMS = STATIC_TRANSFORMERS + STATIC_COL_PARAMS + STATIC_PARAMS
    IRRELEVANT_PARAM_REGEX = re.compile("|".join(IRRELEVANT_PARAMS))
    BASE_PARAM_VALUE_TYPES = [str, int, float, list]

    # 'pipeline' means we're keeping underlying model params rather than params from the wrapping class.
    # We check the value type to avoid logging higher-level params like the model class instance
    # or top-level pipeline object
    return 'pipeline' in key and not re.search(IRRELEVANT_PARAM_REGEX, key) and type(value) in BASE_PARAM_VALUE_TYPES

def present_model_params(model):                           
    return {k: v for k, v in model.get_params().items() if _is_relevant_param(k, v)}

In [16]:
present_model_params(se)

{'pipeline__xgbregressor_meta__base_score': 0.5,
 'pipeline__xgbregressor_meta__booster': 'gbtree',
 'pipeline__xgbregressor_meta__colsample_bylevel': 1,
 'pipeline__xgbregressor_meta__colsample_bynode': 1,
 'pipeline__xgbregressor_meta__colsample_bytree': 1,
 'pipeline__xgbregressor_meta__gamma': 0,
 'pipeline__xgbregressor_meta__importance_type': 'gain',
 'pipeline__xgbregressor_meta__learning_rate': 0.1,
 'pipeline__xgbregressor_meta__max_delta_step': 0,
 'pipeline__xgbregressor_meta__max_depth': 3,
 'pipeline__xgbregressor_meta__min_child_weight': 1,
 'pipeline__xgbregressor_meta__n_estimators': 100,
 'pipeline__xgbregressor_meta__n_jobs': 1,
 'pipeline__xgbregressor_meta__objective': 'reg:squarederror',
 'pipeline__xgbregressor_meta__random_state': 42,
 'pipeline__xgbregressor_meta__reg_alpha': 0,
 'pipeline__xgbregressor_meta__reg_lambda': 1,
 'pipeline__xgbregressor_meta__scale_pos_weight': 1,
 'pipeline__xgbregressor_meta__subsample': 1,
 'pipeline__xgbregressor_sub__base_score

In [19]:
X_test, y_test = data.test_data()
y_pred = se.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mae

27.93691592839893

In [25]:
def predicted_home_margins(X, pred_col):
    home_teams_df = df.query('at_home == 1').sort_index()
    away_teams_df = (
        df
        .query('at_home == 0')
        .set_index(['oppo_team', 'year', 'round_number'])
        .rename_axis([None, None, None])
        .sort_index()
    )
    
    home_margin_multiplier = (
        (home_teams_df[pred_col] > away_teams_df[pred_col])
        .map(lambda x: 1 if x else -1)
    )
    
    return pd.Series(
        ((home_teams[pred_col].abs() + away_teams[pred_col].abs()) / 2) *
        home_margin_multiplier
    ).reindex(home_teams.index).sort_index()


def calculate_home_margin(home_margin, away_margin):
    # We want True to be 1 and False to be -1
    home_margin_multiplier = ((home_margin > away_margin).astype(int) * 2) - 1

    return pd.Series(
        ((home_margin.abs() + away_margin.abs()) / 2) *
        home_margin_multiplier
    ).reindex(home_margin.index).sort_index()


def match_accuracy(X, y_true, y_pred):
    team_match_data_frame = X.assign(y_true=y_true, y_pred=y_pred)
    home_match_data_frame = team_match_data_frame.query("at_home == 1").sort_index()
    away_match_data_frame = (
        team_match_data_frame
        .query('at_home == 0')
        .set_index(['oppo_team', 'year', 'round_number'])
        .rename_axis([None, None, None])
        .sort_index()
    )
    
    home_margin = calculate_home_margin(
        home_match_data_frame['y_true'], away_match_data_frame['y_true']
    )
    home_pred_margin = calculate_home_margin(
        home_match_data_frame['y_pred'], away_match_data_frame['y_pred']
    )
    
    return (
        # Any zero margin (i.e. a draw) is counted as correct per usual tipping rules.
        # Predicted margins should never be zero, but since we don't want to encourage
        # any wayward models, we'll count a predicted margin of zero as incorrect
        ((home_margin >= 0) & (home_pred_margin > 0)) |
        ((home_margin <= 0) & (home_pred_margin < 0))
    ).mean()

In [26]:
m_acc = match_accuracy(X_test, y_test, y_pred)
m_acc

0.7028985507246377

In [32]:
with mlflow.start_run():
    mlflow.log_params(present_model_params(se))    
    mlflow.log_param('test_years', data.test_years)
        
    mlflow.log_metric('mae', mean_absolute_error(y_test, y_pred))
    mlflow.log_metric('match_accuracy', match_accuracy(X_test, y_test, y_pred))
    
    mlflow.set_tag('model', 'stacking_estimator')